In [13]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

In [14]:
main_link = 'https://www.airbnb.com/s/New-York--NY--United-States/homes'
items = '?items_offset='
listings = []
types_list = []
ratings_list = []
price_list = []
url_list = []
for i in range(0, 40, 20):
    try:
        link = main_link + items + str(i)
        print(link)
        driver = None
        driver = webdriver.Chrome('/usr/local/bin/chromedriver')
        driver.get(link)
        time.sleep(5)
        listing_urls = driver.find_elements(By.CLASS_NAME, '_mm360j')
        curr_listings = driver.find_elements(By.CLASS_NAME, '_1whrsux9')
        type_list = driver.find_elements(By.CLASS_NAME, '_3c0zz1')
        star_list = driver.find_elements(By.CLASS_NAME, '_10fy1f8')
        cost_list = driver.find_elements(By.CLASS_NAME, '_tyxjp1')
        if len(curr_listings) > 0 and len(type_list) > 0:
            listings += [place.text for place in curr_listings]
            types_list += [types.text for types in type_list]
            ratings_list += [rating.text for rating in star_list]
            price_list += [price.text for price in cost_list]
            url_list += [url.get_attribute('href') for url in listing_urls]
    except Exception as e:
        pass

https://www.airbnb.com/s/New-York--NY--United-States/homes?items_offset=0


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  del sys.path[0]


https://www.airbnb.com/s/New-York--NY--United-States/homes?items_offset=20


In [26]:
listing_review_ratings = []
for url in url_list[:10]:
    driver = None
    driver = webdriver.Chrome('/usr/local/bin/chromedriver')
    driver.get(url)
    time.sleep(3)
    reviews = driver.find_elements(By.CLASS_NAME, '_162hp8xh')
    rating = driver.find_element(By.CLASS_NAME, '_1ne5r4rt')
    name = driver.find_element(By.CLASS_NAME, '_fecoyn4')
    listing_data = {
        'name': name.text,
        'rating': rating.text,
        'reviews': [r.text.split('\n')[-1] for r in reviews]
    }
    listing_review_ratings.append(listing_data)
listing_review_ratings

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


[{'name': "Billie's holiday (includes maid & laundry)",
  'rating': '',
  'reviews': ['I had an awesome time staying at Josh’s and they are both so attentive and helpful for anything you may need. Great area/ location. Would definitely come back!']},
 {'name': "Hell's Kitchen Enclave: a Diamond in the Rough.",
  'rating': '4.59',
  'reviews': ["Michael's place was great! Great value, exactly as advertised, comfortable, had everything we needed, and good location. I would definitely stay here again!",
   'Easy check in, great location, super close to central park and times square! Well equipped with everything you could want for a cozy getaway. Highly recommended! Would love to stay again.',
   'An excellent location (Near the harbor, good restaurants, and Times Square). Micheal’s place is just as advertised. We were thankful for the cleanliness of the apartment as well as the neighborhood. We felt safe at all times. Just a heads up if you are over 60 and don’t do well on narrow stairs 